## Data Ingestion

In [29]:
from collections import namedtuple # gives the flexibility of combining tuple + dict + lightweight
import os

DataIngestionConfig = namedtuple("DataIngestionConfig", 
    [
        "root_dir",
        "source_url",
        "local_data_files",
        "unzip_dir",
    ])

In [33]:
%pwd

'c:\\Users\\Bijay\\Downloads\\DJANGO_PROJECTS\\ML\\dl_project'

In [ ]:
# os.chdir("dl_project")

In [34]:
%pwd

'c:\\Users\\Bijay\\Downloads\\DJANGO_PROJECTS\\ML\\dl_project'

In [35]:
# from dataclasses import dataclass
# from pathlib import Path

# @dataclass(frozen=True)
# class DataIngestionConfig:
#     root_dir: Path
#     source_url: str
#     local_data_files: Path
#     unzip_dir: Path

In [36]:
from cnnClassifier.constants import *
from cnnClassifier.utils import read_yaml, create_dir

In [37]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, param_filepath = PARAM_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(param_filepath)

        create_dir([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        
        config = self.config.data_ingestion

        data_ingestion_config = DataIngestionConfig(
            root_dir =  config.root_dir,
            source_url = config.source_url,
            local_data_files = config.local_data_files,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [38]:
import urllib.request as request
from zipfile import ZipFile

In [44]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        # os.makedirs(os.path.dirname(self.config.local_data_files), exist_ok=True)
        
        if not os.path.exists(self.config.local_data_files):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename= self.config.local_data_files
            )

    # _sth are hidden methods from the user that developers can only access
    
    def _get_updated_list_of_files(self, list_of_files):
        return [f for f in list_of_files if f.endswith('.jpg') and ('Cat' in f or 'Dog' in f)]
    
    def _preprocess(self, zf: ZipFile, f: str, working_dir: str):
        target_filepath = os.path.join(working_dir, f)
        if not os.path.exists(target_filepath):
            zf.extract(f, working_dir)
        
        if os.path.getsize(target_filepath) == 0:
            os.remove(target_filepath)
    
    def unzip_n_clean(self):
        with ZipFile(file = self.config.local_data_files, mode='r') as zf:
            list_of_files = zf.namelist()
            updated_list = self._get_updated_list_of_files(list_of_files)

            for f in updated_list:
                self._preprocess(zf, f, self.config.unzip_dir)    

In [45]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_n_clean()

except Exception as e:
    raise e

[2025-09-07 23:33:58,732: INFO: common_cmd: Yaml file : config\config.yaml loaded successfully]
[2025-09-07 23:33:58,735: INFO: common_cmd: Yaml file : params.yaml loaded successfully]
[2025-09-07 23:33:58,736: INFO: common_cmd: Directory created succesfully at artifacts]
